In [5]:
# Huggingface token for meta licence acceptance
from dotenv import load_dotenv
import os

load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

True

In [4]:
import lightning as L
from torch import optim
from transformers import AutoModelForCausalLM

class LitLLM(L.LightningModule):
    def __init__(self, llm):
        super().__init__()
        self.llm = llm
    
    def training_step(self, batch, batch_idx):
        outputs = self.llm(**batch)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=5e-5)
    
    def validation_step(self, batch, batch_idx):
        outputs = self.llm(**batch)
        self.log("val_loss", outputs.loss)
    
model = LitLLM(
    AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=HF_TOKEN)
)
        

/Users/echapman/projects/llm-finetuning/.venv/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.62s/it]
